In [2]:
import polars as pl
import numpy as np
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()
output_file = os.getenv('EPC_OUTPUT_FILE')
all_epc = pl.read_csv(
    output_file,
    try_parse_dates=True
    #dtype={'transaction-type': str, 'aircon-present': str},
    #parse_dates=['lodgement-date', 'lodgement-datetime']
)
# len(all_epc) # 1378713 before any filtering
# remove small properties
all_epc = all_epc.filter(pl.col('floor-area') >= 40) # 40m2 min size
# and remove prior records for same property
dedup = all_epc.sort('lodgement-datetime').unique(subset=['building-reference-number'], keep='last')
# df.sort_values('DATE_CHANGED').drop_duplicates('STATION_ID',keep='last')
len(dedup) # 1213676 (1281342 before filter by floor area)

1217573

In [6]:
len(dedup.filter(pl.col("uprn").is_null())) # 440164

440164